In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [31]:
month = '02'
path = f'/Users/bimajenie/Desktop/Hackathon/sample_s3/v0.2/daily-24h/v0.1.2/US/quadkey/total/2020/{month}'

directory = os.path.join("c:\\",path)
quadkey = '0302332'

li = []
cnt = 0
for root,dirs,files in os.walk(directory):
    for day in dirs:
        full_path = (f'{os.path.join(root, day)}/data/{quadkey}.csv')
        try:
            # print(f'Path: {full_path}')
            data = pd.read_csv(full_path, index_col = None, header = 0)
            # print(f'Day {day} **')
            # print(data)
            li.append(data)
        except FileNotFoundError:
            # print(f'{full_path} not found, expected 1 exception')
            pass

df = pd.concat(li, axis=0, ignore_index=True)



In [73]:
spec_quadkey = '3023321213'
df['geography'] = df['geography'].astype(str)

print(df.dtypes)
df = df[df['geography'].apply(lambda s: s.startswith(spec_quadkey))]


display(df.head())
display(df.tail())

print(f'Count: {df.index}')

geography                object
xlon                    float64
xlat                    float64
bounds                   object
agg_day_period           object
activity_index_total    float64
dtype: object


,geography,xlon,xlat,bounds,agg_day_period,activity_index_total
16,30233212133212332,-71.072617,42.310323,"-71.07330322265625,42.30981541568666,-71.07192...",2020-02-03,0.060260
28,30233212131322130,-71.050644,42.364125,"-71.05133056640625,42.363617631805496,-71.0499...",2020-02-03,0.122739
47,30233212131212011,-71.076736,42.382387,"-71.07742309570312,42.3818796109132,-71.076049...",2020-02-03,0.356703
56,30233212131213223,-71.068497,42.375286,"-71.06918334960938,42.37477836111418,-71.06781...",2020-02-03,0.184957
59,30233212130322213,-71.142654,42.361081,"-71.14334106445312,42.360573452384564,-71.1419...",2020-02-03,0.020612


,geography,xlon,xlat,bounds,agg_day_period,activity_index_total
2245296,30233212133122110,-71.050644,42.333677,"-71.05133056640625,42.33316920061984,-71.04995...",2020-02-25,0.024742
2245327,30233212133200002,-71.102829,42.324539,"-71.103515625,42.3240317953547,-71.10214233398...",2020-02-25,0.035196
2245349,30233212130111023,-71.112442,42.419908,"-71.11312866210938,42.41940144722477,-71.11175...",2020-02-25,0.054290
2245362,30233212132231030,-71.155014,42.307276,"-71.15570068359375,42.30676863078422,-71.15432...",2020-02-25,0.044223
2245390,30233212132010310,-71.160507,42.353977,"-71.16119384765625,42.353469793490646,-71.1598...",2020-02-25,0.067837


Count: Int64Index([     16,      28,      47,      56,      59,      72,      86,
                 90,      99,     107,
            ...
            2245231, 2245247, 2245248, 2245251, 2245258, 2245296, 2245327,
            2245349, 2245362, 2245390],
           dtype='int64', length=190916)


In [74]:
"""
quadkey-month.json


{
    "type": "FeatureCollection",
    "features": []

}

{
    "type": "Feature",
    "geometry": {
       "type": "Point",
       "coordinates":  [ -122.30873107910156,47.58995720646247 ]
    },
    "properties": {
    "ActivityIndex":0.003514
    }
  }
"""

'\nquadkey-month.json\n\n\n{\n    "type": "FeatureCollection",\n    "features": []\n\n}\n\n{\n    "type": "Feature",\n    "geometry": {\n       "type": "Point",\n       "coordinates":  [ -122.30873107910156,47.58995720646247 ]\n    },\n    "properties": {\n    "ActivityIndex":0.003514\n    }\n  }\n'

In [75]:
import json

data = {
    "type": "FeatureCollection",
    "features": []
}

In [76]:
cnt = 0
for index,row in df.iterrows():
    cnt+=1
    if(cnt%50000 == 0):
        print(cnt)
    geo_obj = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [None,None]
        },
        "properties": {
            
        }
    }
    geo_obj["properties"]["ActivityIndex"] = row["activity_index_total"]
    geo_obj["properties"]["DayPeriod"] = row["agg_day_period"]
    geo_obj["properties"]["Geography"] = row["geography"]
    
    geo_obj["geometry"]["coordinates"][0] = row["xlon"]
    geo_obj["geometry"]["coordinates"][1] = row["xlat"]
    
    #print(geo_obj)
    data["features"].append(geo_obj)

# json.dump(data, indent=4)

50000
100000
150000


In [77]:
fname = f'{spec_quadkey}-{month}.json'
with open(fname, 'w') as outfile:
    json.dump(data, outfile)